In [ ]:
#Step 1:Insert Data from polygon

In [2]:
!pip install polygon-api-client pandas matplotlib requests python-dotenv

# Set up Polygon API Environment
import os
import pandas as pd
import requests
from datetime import datetime, timedelta
from dotenv import load_dotenv
import time


load_dotenv()
API_KEY = os.getenv("xxxxxxxxx") 
if not API_KEY:
    API_KEY = "xxxxxxxxx"  # Import your API Key

#Define target currency
crypto_pairs = {
    'BTC/USDT': 'X:BTCUSDT',
    'ETH/USDT': 'X:ETHUSDT',
    'USDT/USD': 'X:USDTUSD',
    'BNB/USDT': 'X:BNBUSDT',
    'XRP/USDT': 'X:XRPUSDT'
}

#Set time horizion
start_date = "2023-01-01"  
end_date = "2024-01-01"    
timespan = "minute"          

In [3]:
##Data install function
def get_crypto_data(ticker, start, end, timespan="hour", limit=1000):
    url = f"https://api.polygon.io/v2/aggs/ticker/{ticker}/range/1/{timespan}/{start}/{end}"
    params = {
        "apiKey": API_KEY,
        "limit": limit,
        "adjusted": "true"
    }
    
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        if 'results' in data and data['results']:
            df = pd.DataFrame(data['results'])
            df = df.rename(columns={
                't': 'timestamp',
                'o': 'open',
                'h': 'high',
                'l': 'low',
                'c': 'close',
                'v': 'volume'
            })
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            return df
        else:
            print(f"No data found for {ticker}")
            return pd.DataFrame()
    else:
        print(f"Error fetching data for {ticker}: {response.status_code}")
        print(response.text)
        return pd.DataFrame()



In [26]:
# Data install
crypto_data = {}
for symbol, ticker in crypto_pairs.items():
    print(f"Fetching data for {symbol} ({ticker})...")
    df = get_crypto_data(ticker, start_date, end_date, timespan)
    if not df.empty:
        crypto_data[symbol] = df
        print(f"Retrieved {len(df)} records for {symbol}")
    else:
        print(f"No data retrieved for {symbol}")
    time.sleep(1)


for symbol, df in crypto_data.items():
    print(f"\nData for {symbol}:")
    print(df.head())
    
for symbol, df in crypto_data.items():
    filename = f"{symbol.replace('/', '_')}_polygon_data.csv"
    df.to_csv(filename, index=False)
    print(f"Saved data for {symbol} to {filename}")

Fetching data for BTC/USDT (X:BTCUSDT)...
No data found for X:BTCUSDT
No data retrieved for BTC/USDT
Fetching data for ETH/USDT (X:ETHUSDT)...
No data found for X:ETHUSDT
No data retrieved for ETH/USDT
Fetching data for USDT/USD (X:USDTUSD)...
Retrieved 1000 records for USDT/USD
Fetching data for BNB/USDT (X:BNBUSDT)...
No data found for X:BNBUSDT
No data retrieved for BNB/USDT
Fetching data for XRP/USDT (X:XRPUSDT)...
No data found for X:XRPUSDT
No data retrieved for XRP/USDT

Data for USDT/USD:
         volume      vw     open    close     high      low  \
0  2.546498e+05  0.9997  0.99976  0.99973  0.99980  0.99972   
1  1.045276e+06  0.9999  0.99973  0.99982  1.00001  0.99972   
2  2.540014e+05  0.9998  0.99982  0.99978  0.99986  0.99978   
3  4.916315e+05  0.9998  0.99978  0.99973  0.99986  0.99973   
4  5.655197e+05  0.9998  0.99973  0.99974  0.99986  0.99973   

            timestamp    n  
0 2024-01-01 00:00:00  341  
1 2024-01-01 00:01:00  415  
2 2024-01-01 00:02:00  351  
3 2